In [ ]:
!pip install boto3

In [ ]:
AWS_KEY_ID = '*******'
AWS_SECRET_KEY = '**********'

IMAGE_PATH_ON_S3 = 'dataset_v1_ratio_0.5/content/xmls/1530614630188935288dbe5dd10f2b14ddca3e11b7aeb20d7a5.png'
IMAGE_LOCAL_PATH = '/content/here/hey.png'
S3_BUCKET_NAME = 'airborne-object-detection'

In [ ]:
import boto3
import botocore
import os

In [ ]:
s3 = boto3.client('s3', aws_access_key_id=AWS_KEY_ID, aws_secret_access_key=AWS_SECRET_KEY)

Downloading only one file from s3 bucket

In [34]:
def download_file(bucket_name=S3_BUCKET_NAME, file_local= IMAGE_LOCAL_PATH, s3_path= IMAGE_PATH_ON_S3, client=s3):
    """
    params:
    - bucket_name: s3 bucket name
    - file_local: local path to folder in which to place files along with the file type
    - s3_path: full file path that needs to be download
    - client: initialized s3 client object
    """
    client.download_file( bucket_name, s3_path, file_local)

In [ ]:
download_file()

downloading a directory from s3 bucket

In [ ]:
def download_dir(prefix, local, bucket, client=s3):
    """
    params:
    - prefix: pattern to match in s3
    - local: local path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket':bucket,
        'Prefix':prefix,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    for k in keys:
        dest_pathname = os.path.join(local, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        client.download_file(bucket, k, dest_pathname)

In [ ]:
download_dir("dataset_v1_ratio_0.5/", '/content/here', 'airborne-object-detection', client=s3)